# GEOCODING con GeoPy

    - Autor: alvarofda
    - Fecha: 10/08/2021
    
## Objetivo

    - Trasnformar un set de direcciones a latitud y longitud.
    - Calcular distancia a un punto específico.

## 1 - Observamos la data

In [1]:
# Librerias
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
#import unidecode
import warnings
warnings.filterwarnings("ignore")

In [2]:
# importamos set de datos
df = pd.read_csv('direcciones.csv').drop(columns=['Unnamed: 0'])

In [3]:
# Vemos algunos registros de la data
df.head()

,Dirección,Comuna
0,Manuel Rodríguez 39,Santiago (Centro)
1,Sergio Valdovinos 1399,Santiago (Centro)
2,Huérfanos 1685,Santiago (Centro)
3,Av Fermín vivaceta 2040,Independencia
4,Maule 1467,Renca


In [4]:
# dimensión de la data
df.shape

(238, 2)

In [5]:
# datos nulos
df.isnull().sum()

Dirección     1
Comuna        1
dtype: int64

In [6]:
# Borramos los datos NaN
df.dropna(inplace=True)

## 2.- Un poco de limpieza

En el nombre de las columnas quitamos los espacios y cambiamos minúsculas por mayúsculas.

In [7]:
# En los nombres de columnas cambiamos espacios por sin espacio y mayusculas las cambiamos por minusculas
df.columns = df.columns.str.replace(" ", "")
df.columns = df.columns.str.lower()

In [8]:
df.head()

,dirección,comuna
0,Manuel Rodríguez 39,Santiago (Centro)
1,Sergio Valdovinos 1399,Santiago (Centro)
2,Huérfanos 1685,Santiago (Centro)
3,Av Fermín vivaceta 2040,Independencia
4,Maule 1467,Renca


## 3.- Preprocesamiento de dirección

Generamos una nueva columna la cual contenga la dirección de manera más específica. Esto lo logramos concatenando la columnas dirección, comuna y agregando el pais Chile.

In [9]:
df['direccion_esp'] = df['dirección'] + "," + " " + df['comuna'] + ","+ " " + 'Chile' 

Ademas quitamos algunos caracteres y palabras que nos pueden ensuciar la busqueda en la Api

In [10]:
df['direccion_esp'] = df['direccion_esp'].str.replace("(", "")
df['direccion_esp'] = df['direccion_esp'].str.replace(")", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("Centro", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("Calle", "")
df['direccion_esp'] = df['direccion_esp'].str.replace("calle", "")
#df['direccion_esp'] = df['direccion_esp'].str.replace("de", "")
#df['direccion_esp'] = df['direccion_esp'].str.replace("del", "")
#df['direccion_esp'] = df['direccion_esp'].str.replace("De", "")
#df['direccion_esp'] = df['direccion_esp'].str.replace("Del", "")

In [11]:
# Observamos como queda nuestro df
df.head()

,dirección,comuna,direccion_esp
0,Manuel Rodríguez 39,Santiago (Centro),"Manuel Rodríguez 39, Santiago , Chile"
1,Sergio Valdovinos 1399,Santiago (Centro),"Sergio Valdovinos 1399, Santiago , Chile"
2,Huérfanos 1685,Santiago (Centro),"Huérfanos 1685, Santiago , Chile"
3,Av Fermín vivaceta 2040,Independencia,"Av Fermín vivaceta 2040, Independencia, Chile"
4,Maule 1467,Renca,"Maule 1467, Renca, Chile"


## 4.- Transformación de dirección a Latitud y Longitud

Ahora procedemos a "jugar" con la biblioteca GeoPy. Primero generamos nuestro objeto "geocoder". Aqui utilizamos el metodo "Nominatum".
Según la documentacion de GeoPy, para realizar las consultas a la API debe haber un retardo mínimo de un segundo. Para esto utilizamos la función "RateLimiter" la cual nos permitira realizar consultas con un 1s de delay entre consultas.

In [12]:
# Nominatim geocoder for OpenStreetMap data with RateLimiter
geocoder = RateLimiter(Nominatim(user_agent='MyApp2').geocode, min_delay_seconds=1)

Una vez instanciado nuestro elemento geocoder, prodecemos a entregarle nuestra data. A este objeto podemos solicitarle que nos entregue directamente la longitud y la latitud con .latitude o .longitude pero para esto habria que hacer dos llamados a la api y eso toma el doble de tiempo. Por eso opté por ocupar el metodo .raw para obtener toda la informacion de la direccion de una vez luego y despues filtrarla.

In [13]:
%%time
raw_list =[]

for idx, value in df['direccion_esp'].iteritems():
    try:
        raw = geocoder(value).raw
        raw_list.append(raw)
    except:
        raw_list.append('No data')
        pass
    

Wall time: 3min 57s


In [14]:
# Vemos que es lo que guardo raw_list
raw_list[:2]

[{'place_id': 191661034,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 445535001,
  'boundingbox': ['-33.4565161', '-33.4557998', '-70.6574126', '-70.6570149'],
  'lat': '-33.456178',
  'lon': '-70.6571672',
  'display_name': 'Manuel Rodríguez, Santiago, Provincia de Santiago, Región Metropolitana de Santiago, 8370403, Chile',
  'class': 'highway',
  'type': 'primary',
  'importance': 0.5199999999999999},
 {'place_id': 110149353,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 89908536,
  'boundingbox': ['-33.4324663', '-33.4280714', '-70.707171', '-70.7064054'],
  'lat': '-33.4308179',
  'lon': '-70.7069057',
  'display_name': 'Sergio Valdovinos, Quinta Normal, Provincia de Santiago, Región Metropolitana de Santiago, WW, Chile',
  'class': 'highway',
  'type': 'secondary',
  'importance': 0.51}]

Observamos en que posición de la data obtenemos los datos que necesitamos

In [15]:
# Latitud
print(raw_list[0]['lat'])
# longitud
print(raw_list[0]['lon'])

-33.456178
-70.6571672


Procedemos a obtener la latitud y la longitud de toda la data.

In [16]:
lat = []
lon = []
for data in raw_list:  
    if data != 'No data':
        lat.append(data['lat'])
        lon.append(data['lon'])
    else:
        lat.append('No data')
        lon.append('No data')

Generamos dos series y la concatenamos al df

In [17]:
df['latitud'] = pd.Series(lat)
df['longitud'] = pd.Series(lon)

In [18]:
df.sample(5)

,dirección,comuna,direccion_esp,latitud,longitud
56,General Freire 5011,Renca,"General Freire 5011, Renca, Chile",-33.4009058,-70.7242124
136,Pasaje pullay 5825,Renca,"Pasaje pullay 5825, Renca, Chile",-33.4071094,-70.7345343
134,San Francisco 12,Santiago (Centro),"San Francisco 12, Santiago , Chile",-33.44348590645161,-70.64699593548387
176,Manuel Rodríguez 1358,Renca,"Manuel Rodríguez 1358, Renca, Chile",-33.4015078,-70.72111
109,El olivo 1602,Conchalí,"El olivo 1602, Conchalí, Chile",-33.3903291,-70.6717221


Verificamos cuantas direcciones NO se encontraron

In [19]:
df['latitud'].value_counts()

No data                51
-33.4494469             8
-33.3903291             8
-33.4141433             6
-33.437259147619045     5
                       ..
-33.42381925            1
-33.4009058             1
-33.4055961             1
-33.3996419             1
-33.2107139             1
Name: latitud, Length: 118, dtype: int64

Podemos observar que 71 direcciones no fueron encontradas. El gran motivo es porque las direcciones vienen incompletas o bien, estan mal escritas. Me falta probar con la api de google. Creo que podria terner un mejor rendimiento pero este servicio es pagado :(.

## 5.- Calcular distancia a una dirección específica

Ahora procedemos a calcular la distancia a una direccion específica. Para esto le entregamos nuestra dirección especifica a geocoder para que la convierta a latitud y longitud. En este caso utilizaremos la dirección 'Dublé Almeyda 3143, Ñuñoa, Chile'.

In [20]:
coor_origen = (geocoder('Dublé Almeyda 3143, Ñuñoa, Chile').latitude, geocoder('Dublé Almeyda 3143, Ñuñoa, Chile').longitude)

In [21]:
coor_origen

(-33.4566638, -70.5984268)

Iteramos sobre las columnas del dataframe y calculamos la distancia con el método .distance donde se le entrega el par de coordeadas de destino y el par de cordenadas de origen.

In [22]:
distancia = []
for lat, lon in zip(df['latitud'], df['longitud']):
    if lat != 'No data':
        d = distance.distance((lat, lon), (coor_origen[0], coor_origen[1])).km
        distancia.append(d)
    else:
        distancia.append('No data')

Generamos una nueva columna con las distancia y la concatenamos a nuestro df

In [23]:
df['distancia'] = pd.Series(distancia)

Observamos nuestro dataframe final

In [24]:
df.sample(10)

,dirección,comuna,direccion_esp,latitud,longitud,distancia
30,Marcos mawada 1971,Independencia,"Marcos mawada 1971, Independencia, Chile",No data,No data,No data
46,Lord Cochrane 209,Santiago (Centro),"Lord Cochrane 209, Santiago , Chile",-33.4494469,-70.65516594311204,5.33557
97,José Manuel Borgoño 2974,Renca,"José Manuel Borgoño 2974, Renca, Chile",-33.41362003673469,-70.68498490612245,9.35849
31,Herrera 418,Santiago (Centro),"Herrera 418, Santiago , Chile",-33.44113886363636,-70.67777206363635,7.57558
19,Pasaje pullay 5025,Quinta Normal,"Pasaje pullay 5025, Quinta Normal, Chile",No data,No data,No data
224,El olivo 1602,Conchalí,"El olivo 1602, Conchalí, Chile",-33.3903291,-70.6717221,10.0299
168,Av cerrillos 8220,Lo Espejo,"Av cerrillos 8220, Lo Espejo, Chile",No data,No data,No data
207,Maruri 325,Independencia,"Maruri 325, Independencia, Chile",-33.4281108877551,-70.65550140612245,6.1801
105,Nataniel cox,Santiago (Centro),"Nataniel cox, Santiago , Chile",-33.4454293,-70.654042,5.31883
154,Elvira Dávila 4568,Quinta Normal,"Elvira Dávila 4568, Quinta Normal, Chile",-33.4427874,-70.6921399,8.84797


In [26]:
df[df['latitud']=='No data']

,dirección,comuna,direccion_esp,latitud,longitud,distancia
5,José Manuel borgoy,Renca,"José Manuel borgoy, Renca, Chile",No data,No data,No data
7,Conde de Maule 4106,Estación Central,"Conde de Maule 4106, Estación Central, Chile",No data,No data,No data
8,Fernando santiban 4247,Recoleta,"Fernando santiban 4247, Recoleta, Chile",No data,No data,No data
9,Mawin 802,Independencia,"Mawin 802, Independencia, Chile",No data,No data,No data
12,Av dorsol 1301,Conchalí,"Av dorsol 1301, Conchalí, Chile",No data,No data,No data
19,Pasaje pullay 5025,Quinta Normal,"Pasaje pullay 5025, Quinta Normal, Chile",No data,No data,No data
30,Marcos mawada 1971,Independencia,"Marcos mawada 1971, Independencia, Chile",No data,No data,No data
32,Ismael Valdez 1954,Pedro Aguirre Cerda,"Ismael Valdez 1954, Pedro Aguirre Cerda, Chile",No data,No data,No data
38,Padre Vicente irrazanal 556,Estación Central,"Padre Vicente irrazanal 556, Estación Central,...",No data,No data,No data
50,Lord conchane 209,Santiago (Centro),"Lord conchane 209, Santiago , Chile",No data,No data,No data
